## Keras to ONNX conversion

keras2onnx suports both the SavedModel format and the older HDF5 (.h5 or .keras) format. 
It can convert multi-backend keras as well as tf.keras models.  
Note: tf.keras is also supported by the tensorflow to onnx converter, see [this notebook](./tensorflow.ipynb).

In [1]:
import os

import numpy as np
import keras
import keras2onnx

import onnx
import onnxruntime as ort

## Download and initialize built-in model

In [2]:
# Note: a ~100MB file will be downloaded
model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet')

## Evaluate model on some random input

In [3]:
input_shape = [1] + model.inputs[0].shape[1:]  # input shape without a 1 for batch size, instead of None
input_data = np.random.normal(size=input_shape).astype(np.float32)
pred = model.predict(input_data)

## Convert to ONNX

In [4]:
# Convert the model to ONNX
fname = 'resnet.onnx'

# Note: convert to ONNX and storing to disk is not working with tensorflow 2.3 nor 2.4 as backend
# Workaround (only working with TF 2.3, not 2.4): convert model to ONNX protobuf, then save to disk with ONNX

# keras2onnx.save_model(model, fname)  # this is not working
onnx_protobuf = keras2onnx.convert_keras(model, model.name)
onnx.save(onnx_protobuf, fname)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 458 -> 127


## Evaluate ONNX models and compare to tensorflow output

In [5]:
# verify the ONNX model is valid
onnx_model = onnx.load(fname)
onnx.checker.check_model(onnx_model)

# get ONNX predictions
sess = ort.InferenceSession(fname)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

onnx_input = {input_name: input_data}
pred_onnx = sess.run([output_name], onnx_input)[0]

print(np.allclose(pred_onnx, pred, atol=1e-5))

True
